# Bitcoin Forecasting Prompt Generation (Ollama Example)

This notebook demonstrates how to transform historical BTC/USDT price data into prompt–completion pairs that can be used for **fine-tuning or prompting a large language model (LLM)**.

It aligns with the project goal of using Ollama to **forecast short-term price movements** based on past trends.

We will:

1. Load historical BTC closing prices
2. Construct prompts from the past 60 minutes
3. Attach the next closing price as the target (completion)
4. Format and write to a `.jsonl` file for training or testing

We do **not** actually run the LLM in this notebook — instead, we prepare high-quality inputs **for fine-tuning** or **prompt-response evaluation**.

In [1]:
import json
from pathlib import Path
import pandas as pd

## Step 1 – Load Historical Data

We use a historical BTC/USDT CSV file exported from Binance or another exchange.

Assumptions:
- The CSV file is located at `historical_analysis/data/historical_btc_data.csv`
- It has a `start` timestamp column (in milliseconds)
- It has a `close` column for the closing price

In [2]:
# Constants
INPUT_CSV = Path("data/fresh_btc_data.csv")
WINDOW = 60  # Use last 60 closes as context

# Load CSV
df = pd.read_csv(INPUT_CSV)
df["Time"] = pd.to_datetime(df["timestamp"], utc=True)
series = df.set_index("Time")["close"].astype(float).dropna()

# Preview
series.head()

Time
2024-11-20 00:00:00+00:00    92251.652407
2024-11-20 00:05:00+00:00    92251.652407
2024-11-20 00:10:00+00:00    92251.652407
2024-11-20 00:15:00+00:00    92251.652407
2024-11-20 00:20:00+00:00    92251.652407
Name: close, dtype: float64

## Step 2 – Create Prompts from Price Windows

We'll create a helper function that:
- Takes a list of 60 prices
- Formats them as a numbered list
- Builds a **prompt** string asking the model to forecast the next price

In [7]:
def make_example(prices):
    """
    Given a list of floats, builds a prompt string listing them and a completion
    asking for the next price.
    """
    lines = [f"{i+1:02d}. {p:.2f}" for i, p in enumerate(prices)]
    prompt = (
        "Here are the last 60 BTC/USDT closing prices (most recent last):\n"
        + "\n".join(lines)
        + "\n\nPlease forecast the next closing price (just the number)."
    )
    return prompt

## Step 3 – Generate Prompt–Completion Pairs

For every sequence of 60 prices, we attach the **next closing price** as the label.  
This structure is used for both **prompt-based forecasting** and **fine-tuning**.

We will generate a few samples below and inspect them.

In [8]:
samples = []

for i in range(3):  # Just create 3 examples for preview
    hist = series.iloc[i : i + WINDOW].tolist()
    target = series.iloc[i + WINDOW]
    prompt = make_example(hist)
    completion = f" {target:.2f}"  # space prefix is required
    samples.append({
        "prompt": prompt,
        "completion": completion
    })

samples[0]  # Show first example


{'prompt': 'Here are the last 60 BTC/USDT closing prices (most recent last):\n01. 92251.65\n02. 92251.65\n03. 92251.65\n04. 92251.65\n05. 92251.65\n06. 92251.65\n07. 92251.65\n08. 92251.65\n09. 92251.65\n10. 92251.65\n11. 92251.65\n12. 92251.65\n13. 92251.65\n14. 92251.65\n15. 92251.65\n16. 92251.65\n17. 92251.65\n18. 92251.65\n19. 92251.65\n20. 92251.65\n21. 92251.65\n22. 92251.65\n23. 92251.65\n24. 92251.65\n25. 92251.65\n26. 92251.65\n27. 92251.65\n28. 92251.65\n29. 92251.65\n30. 92251.65\n31. 92251.65\n32. 92251.65\n33. 92251.65\n34. 92251.65\n35. 92251.65\n36. 92251.65\n37. 92251.65\n38. 92251.65\n39. 92251.65\n40. 92251.65\n41. 92251.65\n42. 92251.65\n43. 92251.65\n44. 92251.65\n45. 92251.65\n46. 92251.65\n47. 92251.65\n48. 92251.65\n49. 92251.65\n50. 92251.65\n51. 92251.65\n52. 92251.65\n53. 92251.65\n54. 92251.65\n55. 92251.65\n56. 92251.65\n57. 92251.65\n58. 92251.65\n59. 92251.65\n60. 92251.65\n\nPlease forecast the next closing price (just the number).',
 'completion': ' 922

## Sample Output

This is a single prompt–completion pair that we would send to an LLM or use to fine-tune a forecasting head.

- The **prompt** gives the last 60 BTC closing prices in order.
- The **completion** is a single number: the next price.

This lets the model learn short-term patterns from raw prices.


In [9]:
# Pretty print all three samples
for i, sample in enumerate(samples, 1):
    print(f"\n--- Sample #{i} ---")
    print("Prompt:\n", sample["prompt"])
    print("Completion:\n", sample["completion"])


--- Sample #1 ---
Prompt:
 Here are the last 60 BTC/USDT closing prices (most recent last):
01. 92251.65
02. 92251.65
03. 92251.65
04. 92251.65
05. 92251.65
06. 92251.65
07. 92251.65
08. 92251.65
09. 92251.65
10. 92251.65
11. 92251.65
12. 92251.65
13. 92251.65
14. 92251.65
15. 92251.65
16. 92251.65
17. 92251.65
18. 92251.65
19. 92251.65
20. 92251.65
21. 92251.65
22. 92251.65
23. 92251.65
24. 92251.65
25. 92251.65
26. 92251.65
27. 92251.65
28. 92251.65
29. 92251.65
30. 92251.65
31. 92251.65
32. 92251.65
33. 92251.65
34. 92251.65
35. 92251.65
36. 92251.65
37. 92251.65
38. 92251.65
39. 92251.65
40. 92251.65
41. 92251.65
42. 92251.65
43. 92251.65
44. 92251.65
45. 92251.65
46. 92251.65
47. 92251.65
48. 92251.65
49. 92251.65
50. 92251.65
51. 92251.65
52. 92251.65
53. 92251.65
54. 92251.65
55. 92251.65
56. 92251.65
57. 92251.65
58. 92251.65
59. 92251.65
60. 92251.65

Please forecast the next closing price (just the number).
Completion:
  92251.65

--- Sample #2 ---
Prompt:
 Here are the last

## Step 4 – Write to JSONL (Optional)

This step saves all prompt–completion pairs into a `.jsonl` file named `finetune_data.jsonl`.

This format is compatible with:
- Ollama fine-tuning
- OpenAI fine-tune APIs
- Any system that takes "prompt" → "completion" training pairs

In [10]:
OUTPUT_JSONL = Path("finetune_data.jsonl")

with open(OUTPUT_JSONL, "w") as out:
    for i in range(len(series) - WINDOW):
        hist = series.iloc[i : i + WINDOW].tolist()
        target = series.iloc[i + WINDOW]
        prompt = make_example(hist)
        completion = f" {target:.2f}"
        out.write(json.dumps({
            "prompt": prompt,
            "completion": completion
        }) + "\n")

print(f"Wrote {OUTPUT_JSONL}")

Wrote finetune_data.jsonl


## Summary

- This notebook demonstrates how to prepare time-series data for LLM training or inference.
- The format follows Ollama and OpenAI fine-tune requirements.
- The `.jsonl` output can be used to:
  - Fine-tune a local model (e.g. Mistral, LLaMA)
  - Evaluate prompt performance on holdout data
  - Generate forecasts based on prompt-only workflows

> You can now move to a model script or dashboard and test how the LLM performs on these prompts!